# Apriori
В этом ноутбуке рассматривается вспомогательный метод, который можно применять в дополнение к другим подходам.  
При помощи алгоритма apriori можно находить закономерности в покупках из датасета. Например, определить какой товар покупают часто с другим определенным товаром.  
В дальнейшем этим можно пробовать расширять список рекоммендаций для пользователя.  
Здесь можно найти пояснения для всех параметров и результатов использования этого алгоритма:  
https://stackabuse.com/association-rule-mining-via-apriori-algorithm-in-python

! Ноутбук можно запускать в режиме "Run all"

## Загрузка библиотек

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from efficient_apriori import apriori
tqdm.pandas()

## Подготовка данных

In [2]:
df = pd.read_csv("data/main.csv")

In [3]:
df.rename(columns={"order_completed_at":"time"}, inplace=True) # переименуем колонку для удобства
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%d %H:%M:%S") # изменим тип колонки для дальнейшей работы

In [4]:
df.head()

user_id                time  cart
0        2 2015-03-22 09:25:46   399
1        2 2015-03-22 09:25:46    14
2        2 2015-03-22 09:25:46   198
3        2 2015-03-22 09:25:46    88
4        2 2015-03-22 09:25:46   157

Для пирменения алгоритма необходимо преобразовать корзины в кортежи

In [5]:
carts = df.groupby(["user_id", "time"])["cart"].progress_apply(lambda x: tuple(x)).tolist()

  0%|          | 0/209406 [00:00<?, ?it/s]

Далее найдем закономерности (комплиментарные товары) в покупках

## Поиск закономерностей

In [6]:
itemsets, rules = apriori(carts, min_support=0.1,  min_confidence=0.5)

In [7]:
arr_first = []
arr_second = []
arr_conf = []
arr_supp = []
arr_lift = []

for rule in rules:
    arr_first.append(list(rule.lhs))
    arr_second.append(list(rule.rhs))
    arr_conf.append(rule.confidence)
    arr_supp.append(rule.support)
    arr_lift.append(rule.lift)
    print(rule)

{9} -> {57} (conf: 0.615, supp: 0.106, lift: 1.182, conv: 1.246)
{16} -> {14} (conf: 0.640, supp: 0.119, lift: 1.426, conv: 1.531)
{17} -> {14} (conf: 0.636, supp: 0.179, lift: 1.416, conv: 1.513)
{19} -> {14} (conf: 0.609, supp: 0.132, lift: 1.358, conv: 1.412)
{22} -> {14} (conf: 0.587, supp: 0.192, lift: 1.309, conv: 1.335)
{23} -> {14} (conf: 0.550, supp: 0.189, lift: 1.226, conv: 1.226)
{41} -> {14} (conf: 0.518, supp: 0.109, lift: 1.154, conv: 1.143)
{57} -> {14} (conf: 0.516, supp: 0.268, lift: 1.151, conv: 1.140)
{14} -> {57} (conf: 0.598, supp: 0.268, lift: 1.151, conv: 1.195)
{61} -> {14} (conf: 0.526, supp: 0.230, lift: 1.171, conv: 1.162)
{14} -> {61} (conf: 0.512, supp: 0.230, lift: 1.171, conv: 1.154)
{84} -> {14} (conf: 0.538, supp: 0.176, lift: 1.199, conv: 1.193)
{382} -> {14} (conf: 0.543, supp: 0.122, lift: 1.210, conv: 1.206)
{383} -> {14} (conf: 0.545, supp: 0.127, lift: 1.215, conv: 1.212)
{398} -> {14} (conf: 0.556, supp: 0.217, lift: 1.238, conv: 1.241)
{402} ->

В выведенных закономерностях мы видим конкретные товары, являющиеся предпосылками и следствиями, а также вспомогательную информацию, которая позволяет оценить "уверенность" в правиле.  
Например, `supp` (_support_) - доля покупок, где эти товары были вместе, а `conf (x -> y)` (_confidence_) - вероятность покупки товара `y`, при покупке товара `x`

In [8]:
arr_first[:3]

[[9], [16], [17]]

## Выгрузка результатов

Реструктурируем данные в удобный формат и сохраним основные наблюдения

In [9]:
ap = pd.DataFrame()
ap["from"] = arr_first
ap["to"] = arr_second
ap["conf"] = arr_conf
ap["supp"] = arr_supp
ap["lift"] = arr_lift

In [10]:
ap = ap.sort_values(["supp","conf"], ascending=False).head(20)

In [11]:
ap["from"] = ap["from"].apply(lambda x: x[0])
ap["to"] = ap["to"].apply(lambda x: x[0])

In [12]:
ap.head()

from  to      conf      supp      lift
35    61  57  0.710759  0.310712  1.367021
36    57  61  0.597601  0.310712  1.367021
8     14  57  0.598316  0.268455  1.150757
7     57  14  0.516326  0.268455  1.150757
41   398  57  0.645935  0.251994  1.242343

In [13]:
ap.to_csv("data/apriori_top_20.csv", index=False)